# Reescreve Arquivos XDI

In [1]:
import regex as re
import pandas as pd
import os

___
<font size=3> Este notebook foi criado para adicionar novos campos à arquivos XDI: `Citation.doi` e `Citation.comments`, o primeiro armazenará a referência aos autores do experimento contido no arquivo XDI lido e a referência à base de dados, enquanto o segundo armazenará os comentários sobre o experimento, caso existam. </font>
____

**<font size=3>Objetivo deste notebook</font>**

- <font size=3> Reconhecer onde termina a parte do header </font>

- <font size=3> Reconhecer a parte do comentário, apagar a parte do comentário e adicionar o que estava no comentário em dois metadados novos: `Citation.doi` (referenciando os autores e a base de dados SSHADE) e `Citation.comments` (caso exista algum comentário no arquivo).</font>
___

<font size=3>Devido à grande quantidade de arquivos que obtivemos, fez-se necessário uma rotina que percorra toda a pasta em busca dos arquivos XDI que ela armazena e listá-los em uma lista do python para encontrar os arquivos de modo mais fácil quando a rotina de adição dos novos metadados for executada.</font>

______
<font color='red' size=6>**Atenção!**</font>

<font size=3>Antes de executar os códigos abaixo confira se há necessidade de mudar os valores de algumas variáveis, principalmente `caminho_raiz`, `arquivo_para_leitura`. Além disso, é fundamental observar no arquivo que você quer adicionar os novos campos se há um `doi`, pois do contrário ocorrerá um erro ao executar a função `reconhece_campos`.</font>
_____

In [2]:
#código construído com auxilio do chat GPT

caminho_raiz = r"C:\Users\izaque23003\Downloads\ip3\SSHADE" 

#Criando lista para guardar os nomes das pastas dos dados convertidos
nomes_pastas = []
padrao_arq = re.compile(r'(.+?).xdi$')
arquivo = []
# Percorrendo recursivamente todos os diretórios e subdiretórios
for diretorio_atual, subdiretorios, arquivos in os.walk(caminho_raiz):
    # Iterando sobre os arquivos no diretório atual
    if not nomes_pastas:
        nomes_pastas.extend(subdiretorios)
    for nome_arquivo in arquivos:
        # Verificando se o arquivo corresponde ao padrão .data
        if padrao_arq.search(nome_arquivo):
            # Construindo o caminho completo do arquivo
            caminho_completo = os.path.join(diretorio_atual, nome_arquivo)
            # Adicionando o caminho completo à lista de arquivos .data
            arquivo.append(caminho_completo)

## Encontrando os comentários no arquivo XDI

<font size=3>Para identificar corretamente o campo que será mudado vamos realizar um teste para encontrar um padrão da localização do conteúdo de comentários buscado no arquivo XDI.<font>

In [9]:
arquivo_para_leitura ='Ag K edge XAS fluorescence Ag-GSH 1.xdi'

with open(arquivo_para_leitura, 'r') as arquivo:
    texto = arquivo.readlines()


In [4]:
texto[11] #encontrando trecho comentários

'# ////\n'

In [4]:
texto[11+2] #pulando duas linhas consigo chegar na informação que eu desejo.

'#- Veronesi, Giulia; Delangle, Pascale; Carriere, Marie (2014): Ag K edge XAS fluorescence of Ag-GSH 1:1 solution (pH=7.4) at 16K. SSHADE/FAME (OSUG Data Center). Dataset/Spectral Data. https://doi.org/10.26302/SSHADE/EXPERIMENT_GV_20181121_001\n'

<font size=3> Foi observado que, ao encontrar as informações na posição 11 do arquivo XDI, onde o token de comentário característico dos arquivos XDI (`# ////\n`, veja a documentação oficial em [1]) é encontrado, se avançarmos duas posições na lista `texto` é possível encontrar de fato o comentário que contém os autores que serão referenciados no novo metadado `Citation.doi`</font>

# Função que reconhece os campos

<font size=3>Como há a possibilidade de nem sempre estar na mesma posição, o código abaixo foi escrito de forma a reconhecer, por meio da biblioteca `Regex`, onde inicia o token de comentário e adiciona 2 posições à posição na qual o token foi encontrada, devido ao que foi observado no raciocínio acima.</font>

In [6]:
def reconhece_campos(texto):
    '''Esta função recebe uma lista de strings de arquivo XDI e encontra o token comentário.
    Args:
        texto: lista contendo os dados do arquivo XDI lidos com a função readlines
    returns:
        retorna os campos doi, indice onde inicia o token de comentário, lista contendo os dados dos novos campos e 
        a lista dada como argumento alterada.
    '''
    for linha in texto:
        if re.search(r'#\s?/+\n', linha) != None: #posição de inicio dos comentários
            indice_inicia = texto.index(linha)
            texto[indice_inicia+2] = texto[indice_inicia+2].replace('#- ', '')
            texto[indice_inicia+2] = texto[indice_inicia+2].replace('\n', '')
            texto[indice_inicia+3] = texto[indice_inicia+3].replace('# ', '')
            texto[indice_inicia+3] = texto[indice_inicia+3].replace('\n', '')
            texto[indice_inicia+1] = texto[indice_inicia+1].replace('#', '')
            texto[indice_inicia+1] = texto[indice_inicia+1].replace('\n', '')
            
    novos_campos = [texto.pop(indice_inicia+2)]
    novos_campos.append(texto.pop(indice_inicia+2))
    novos_campos.append(texto.pop(indice_inicia+1))
    
    doi = re.search(r'\s?https://doi.org(.+?)\n?$', novos_campos[0])[0] #localiza o doi do experimento
    
    return indice_inicia, doi, novos_campos, texto
    

In [7]:
indice, doi, campos, texto = reconhece_campos(texto)

# Visualizando os dados obtidos

In [7]:
campos

['Veronesi, Giulia; Delangle, Pascale; Carriere, Marie (2014): Ag K edge XAS fluorescence of Ag-GSH 1:1 solution (pH=7.4) at 16K. SSHADE/FAME (OSUG Data Center). Dataset/Spectral Data. https://doi.org/10.26302/SSHADE/EXPERIMENT_GV_20181121_001',
 "Schmitt, Bernard; Bollard, Philippe; Albert, Damien; Garenne, Alexandre; Bonal, Lydie; and the SSHADE partner's consortium (2018). SSHADE: 'Solid Spectroscopy Hosting Architecture of Databases and Expertise'. OSUG Data Center. Service/Database infrastructure. https://doi.org/10.26302/SSHADE",
 '']

In [8]:
texto[:indice]

['# XDI/1.0\n',
 '# Column.1: energy(ev)\n',
 '# Column.2: fluorescence_emission(au)\n',
 '# Element.symbol: Ag\n',
 '# Element.edge: K\n',
 '# Mono.name: Si 2 2 0\n',
 '# Mono.d_spacing: 1.92\n',
 '# Beamline.name: FAME\n',
 '# Sample.name: Frozen solution of Ag-GSH 1:1 (pH=7.4), model for digonal AgS2 coordination\n',
 '# Sample.temperature: 16.0 +/- 3.0 K\n',
 '# Sample.pressure: None\n']

# Reescrevendo os arquivos

<font size=3>Agora que os dados que serão utilizados para os novos metadados estão separados na variável `campos` e a lista `texto` está alterada, podemos reescrever o nosso arquivo adicionando os novos metadados `Citation.doi` e `Citation.comments`.</font>

In [8]:
def reescreve(arquivo):
    '''Esta funcao recebe um arquivo e reescreve adicionando novos campos.
    Args:
        arquivo: arquivo/caminho do arquivo a ser reescrito.
        
    returns:
        retorna o caminho do arquivo novo para visualização.
    
    '''
    
    new_arq = ''
    caminho_arquivo_novo = ''
    with open(arquivo, "r") as file:
        # Leia todo o conteúdo do arquivo
        novo_conteudo = ''
        conteudo = file.readlines()

        indice, doi, campos, conteudo = reconhece_campos(conteudo)

        conteudo = conteudo[:indice] + [f"# Citation.doi: {doi}"] + ['\n'] + [f"# Citation.comments: {campos[0]}{campos[1]}{campos[2]}"] + ['\n'] + conteudo[indice:]
        for texto in conteudo:
            novo_conteudo += texto
    #encontrando somente o nome do arquivo antes da extensão
    sentenca = r'.*\\([^\\]*\.xdi)'
    new_arq = re.search(sentenca, arquivo)[1]+'_new.xdi'
    novo_caminho = r"C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos"
    caminho_arquivo_novo = os.path.join(novo_caminho,new_arq)
    print(caminho_arquivo_novo)
    #salvando um novo arquivo na pasta com o nome new
    with open(caminho_arquivo_novo, 'w') as fi:
        fi.write(novo_conteudo)
    
    return caminho_arquivo_novo

<font size=3> Com a função para reescrever os arquivos pronta, agora é hora de reescrever todos os arquivos que convertemos adiconando os novos metadados</font>

In [53]:
for file in arquivo:   
    novo_caminho= reescreve(file)
    print(f'O arquivo {file} foi reescrito e salvo na pasta {novo_caminho}')

C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Ag K edge XAS fluorescence Ag-GSH 1.xdi_new.xdi
O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Ag\ Ag K edge XAS fluorescence Ag-GSH 1.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Ag K edge XAS fluorescence Ag-GSH 1.xdi_new.xdi
C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Ag K edge XAS fluorescence of Ag malate solution at 10K.xdi_new.xdi
O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Ag\ Ag K edge XAS fluorescence of Ag malate solution at 10K.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Ag K edge XAS fluorescence of Ag malate solution at 10K.xdi_new.xdi
C:\Users\izaque23003\O

O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Ag\ Ag K edge XAS transmission of uncoated Ag nano-particles at 10K.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Ag K edge XAS transmission of uncoated Ag nano-particles at 10K.xdi_new.xdi
C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Br K edge XAS fluorescence of gallium bromide solution (Ga3+ 0.017m, Br- 0.051m) at 300bar and 300K.xdi_new.xdi
O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Br\ Br K edge XAS fluorescence of gallium bromide solution (Ga3+ 0.017m, Br- 0.051m) at 300bar and 300K.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Br K edge XAS fluorescence of gallium bromide solution (Ga3+ 0.017m, Br- 0.051m) at 300bar and 300K.xdi_ne

C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Br K edge XAS transmission of gallium bromide solution (Ga3+ 0.17m, Br- 0.51m) at 300bar and 420K.xdi_new.xdi
O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Br\ Br K edge XAS transmission of gallium bromide solution (Ga3+ 0.17m, Br- 0.51m) at 300bar and 420K.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Br K edge XAS transmission of gallium bromide solution (Ga3+ 0.17m, Br- 0.51m) at 300bar and 420K.xdi_new.xdi
C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Br K edge XAS transmission of gallium bromide solution (Ga3+ 0.17m, Br- 0.51m) at 300bar and 470K.xdi_new.xdi
O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Br\ Br K edge XAS transmission of gallium bromide solution (Ga3+ 0.17

O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Cd\ Cd K edge XAS transmission of Cadmium acetate dihydrate at 15K.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Cd K edge XAS transmission of Cadmium acetate dihydrate at 15K.xdi_new.xdi
C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Cd K edge XAS transmission of cadmium carbonate $CdCO_3$ at 15K.xdi_new.xdi
O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Cd\ Cd K edge XAS transmission of cadmium carbonate $CdCO_3$ at 15K.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Cd K edge XAS transmission of cadmium carbonate $CdCO_3$ at 15K.xdi_new.xdi
C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos

O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Co\ Co K edge XAS fluorescence of CoFePt deposited on silicium (classical incidence.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Co K edge XAS fluorescence of CoFePt deposited on silicium (classical incidence.xdi_new.xdi
C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Co K edge XAS fluorescence of CoFePt deposited on silicium (grazing incidence.xdi_new.xdi
O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Co\ Co K edge XAS fluorescence of CoFePt deposited on silicium (grazing incidence.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Co K edge XAS fluorescence of CoFePt deposited on silicium (grazing incidence.xdi_new.xdi
C:\Users\izaque23003\OneDriv

...

C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Se K edge XAS HERFD of Se-glutathione peroxydase solution at 10K.xdi_new.xdi
O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Se\ Se K edge XAS HERFD of Se-glutathione peroxydase solution at 10K.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Se K edge XAS HERFD of Se-glutathione peroxydase solution at 10K.xdi_new.xdi
C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Se K edge XAS HERFD of Se-methionine at 10K.xdi_new.xdi
O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Se\ Se K edge XAS HERFD of Se-methionine at 10K.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Se K edge XAS HERFD of Se-m

O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Y\ Y K edge XAS fluorescence of yttrium(III) solution at hydrothermal conditions (Y 0.011m Cl 0.79m HCl 0.75m) at 800 bar and 30Â°C.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Y K edge XAS fluorescence of yttrium(III) solution at hydrothermal conditions (Y 0.011m Cl 0.79m HCl 0.75m) at 800 bar and 30Â°C.xdi_new.xdi
C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Y K edge XAS fluorescence of yttrium(III) solution at hydrothermal conditions (Y 0.011m Cl 0.79m HCl 0.75m) at 800 bar and 350Â°C.xdi_new.xdi
O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Y\ Y K edge XAS fluorescence of yttrium(III) solution at hydrothermal conditions (Y 0.011m Cl 0.79m HCl 0.75m) at 800 bar and 350Â°C.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDrive 

O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Y\ Y K edge XAS fluorescence of yttrium(III) solution at hydrothermal conditions (Y 0.01m Cl 0.03m HCl 0.002m) at 800 bar and 200Â°C.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Y K edge XAS fluorescence of yttrium(III) solution at hydrothermal conditions (Y 0.01m Cl 0.03m HCl 0.002m) at 800 bar and 200Â°C.xdi_new.xdi
C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Y K edge XAS fluorescence of yttrium(III) solution at hydrothermal conditions (Y 0.01m Cl 0.03m HCl 0.002m) at 800 bar and 300Â°C.xdi_new.xdi
O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Y\ Y K edge XAS fluorescence of yttrium(III) solution at hydrothermal conditions (Y 0.01m Cl 0.03m HCl 0.002m) at 800 bar and 300Â°C.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDriv

O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Zn\ Zn K edge XAS HERFD of aqueous Zn(Cysteine)4 at 10K.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Zn K edge XAS HERFD of aqueous Zn(Cysteine)4 at 10K.xdi_new.xdi
C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Zn K edge XAS HERFD of aqueous Zn(Histidine)4 at 10K.xdi_new.xdi
O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Zn\ Zn K edge XAS HERFD of aqueous Zn(Histidine)4 at 10K.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Zn K edge XAS HERFD of aqueous Zn(Histidine)4 at 10K.xdi_new.xdi
C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Zn K edge XAS HERFD of aqueous Zn(Malate)3 at 1

O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Zr\ Zr K edge XAS fluorescence of natural eudialyte Na4(Ca,Ce,Fe,Mn,Y,La,Sr,K)3ZrSi8O22(OH,Cl)2 at ambient conditions.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Zr K edge XAS fluorescence of natural eudialyte Na4(Ca,Ce,Fe,Mn,Y,La,Sr,K)3ZrSi8O22(OH,Cl)2 at ambient conditions.xdi_new.xdi
C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Zr K edge XAS fluorescence of natural vlasovite Na2ZrSi4O11 at ambient conditions.xdi_new.xdi
O arquivo C:\Users\izaque23003\Downloads\ip3\SSHADE\convertidos\Zr\ Zr K edge XAS fluorescence of natural vlasovite Na2ZrSi4O11 at ambient conditions.xdi foi reescrito e salvo na pasta C:\Users\izaque23003\OneDrive - ILUM ESCOLA DE CIÊNCIA\ilum_3s\ip_3\base_dados_fase1\SSHADE\dados_campos_novos\ Zr K edge XAS fluorescence of natural vlas

# Referências Bibliográficas

[1] https://github.com/XraySpectroscopy/XAS-Data-Interchange/blob/master/specification/spec.md#structure-of-the-header-section

